In [ ]:
%matplotlib qt5
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
pd.set_option("display.max_rows",999)
Alpha=0.1 #Learning Rate
Beta=0.9 #The deceasing rate of the future gain
#The rate of choosing a randon action ,Epsilon
Epsilon_start=1
Epsilon_stop=0.01
decay_rate=0.00001
N_Mesh=10
N_States=100 #We assume that the mesh forme is 10*10
Actions=['left_Three','left_Four','left_Six','right_Three','right_Four','right_Six','up_Three','up_Four','up_Six']
Obstacle=[3,5,16,21,23,27,29,31,33,34,56,57,66,67,78,81,86]
#Only the Six_Gait can cross this type of obstacles
Obstacle_Six=[6,7,22,28,44,55,68,82,83,92]
#Only the Six_Gait and Four_Gait can cross this type of obstacles, howerer the Four_Gait is the best
Obstacle_Four=[8,10,11,12,15,24,30,40,45,52,72,79,85]
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Caculate_coordinate(State):
    X_Axis=0.5+(State-1)%N_Mesh
    Y_Axis=0.5+((State-1)//N_Mesh)
    return X_Axis,Y_Axis
#According to the number of the States and Actions that are considered as the variables, we initialize the Q-table
def Initial_Q_Table (N_States,Actions):
    Q_Table=np.zeros([N_States,len(Actions)])
    #We define the colunms labels of Q-table
    Q_Table=pd.DataFrame(Q_Table,index=np.arange(1,N_States+1),columns=Actions)
    print ('We have finished initializing the Q-table') 
    return Q_Table
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
#According to the current state, we choose an action and then execute it.
def Select_Action(State,Q_Table,ep):
    Epsilon=Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*ep)
    States_Actions=Q_Table.loc[[State],:]
    if(np.random.rand()<Epsilon or np.all(States_Actions==[0])):
        Execute_Action=np.random.choice(Actions)#Select an action randomy
    else:
        #Q_T=Q_Table.T
        #G=(Q_Table.T).loc[:,[State]]
        Execute_Action=np.array(pd.DataFrame.idxmax((Q_Table.T).loc[:,[State]]))[0]#Select the action having the maximun QValue
    return Execute_Action
#———————————————————————————————————————————————————————————————————————————————————————————————————————————
#According to the current state and the selected current action, return the new state and rewards
def Next_State(State,Execute_Action):
    if(Execute_Action=='left_Three'):
        if(State%N_Mesh==1):
            State=State
            R=-20
        elif((State-1) in Obstacle):
            State=State
            R=-20
        elif((State-1) in Obstacle_Six):
            State=State
            R=-20
        elif((State-1) in Obstacle_Four):
            State=State
            R=-20
        else:
            State=State-1
            R=1
    elif(Execute_Action=='left_Four'):
        if(State%N_Mesh==1):
            State=State
            R=-20
        elif((State-1) in Obstacle):
            State=State
            R=-20
        elif((State-1) in Obstacle_Six):
            State=State
            R=-20
        elif((State-1) in Obstacle_Four):
            State=State-1
            R=1
        else:
            State=State-1
            R=0.6
    elif(Execute_Action=='left_Six'):
        if(State%N_Mesh==1):
            State=State
            R=-20
        elif((State-1) in Obstacle):
            State=State
            R=-20
        elif((State-1) in Obstacle_Six):
            State=State-1
            R=1
        elif((State-1) in Obstacle_Four):
            State=State-1
            R=0.6
        else:
            State=State-1
            R=0.2
    
    elif(Execute_Action=='right_Three'):
        if(State==N_States-1):
            State='End'
            R=1000
        elif(State%N_Mesh==0):
            State=State
            R=-20
        elif((State+1) in Obstacle):
            State=State
            R=-20
        elif((State+1)in Obstacle_Six):
            State=State
            R=-20
        elif((State+1)in Obstacle_Four):
            State=State
            R=-20
        else:
            State=State+1
            R=1
    elif(Execute_Action=='right_Four'):
        if(State==N_States-1):
            State='End'
            R=1000
        elif(State%N_Mesh==0):
            State=State
            R=-20
        elif((State+1) in Obstacle):
            State=State
            R=-20
        elif((State+1)in Obstacle_Six):
            State=State
            R=-20
        elif((State+1)in Obstacle_Four):
            State=State+1
            R=1
        else:
            State=State+1
            R=0.6
    elif(Execute_Action=='right_Six'):
        if(State==N_States-1):
            State='End'
            R=1000
        elif(State%N_Mesh==0):
            State=State
            R=-20
        elif((State+1) in Obstacle):
            State=State
            R=-20
        elif((State+1)in Obstacle_Six):
            State=State+1
            R=1
        elif((State+1)in Obstacle_Four):
            State=State+1
            R=0.6
        else:
            State=State+1
            R=0.2
    
    elif(Execute_Action=='up_Three'):
        if(State==N_States-N_Mesh):
            State='End'
            R=1000
        elif(N_States+1-N_Mesh<=State<=N_States-1):
            State=State
            R=-20
        elif((State+10) in Obstacle):
            State=State
            R=-20
        elif((State+10) in Obstacle_Six):
            State=State
            R=-20
        elif((State+10) in Obstacle_Four):
            State=State
            R=-20
        else:
            State=State+N_Mesh
            R=1
    elif(Execute_Action=='up_Four'):
        if(State==N_States-N_Mesh):
            State='End'
            R=1000
        elif(N_States+1-N_Mesh<=State<=N_States-1):
            State=State
            R=-20
        elif((State+10) in Obstacle):
            State=State
            R=-20
        elif((State+10) in Obstacle_Six):
            State=State
            R=-20
        elif((State+10) in Obstacle_Four):
            State=State++N_Mesh
            R=1
        else:
            State=State+N_Mesh
            R=0.6
    elif(Execute_Action=='up_Six'):
        if(State==N_States-N_Mesh):
            State='End'
            R=1000
        elif(N_States+1-N_Mesh<=State<=N_States-1):
            State=State
            R=-20
        elif((State+10) in Obstacle):
            State=State
            R=-20
        elif((State+10) in Obstacle_Six):
            State=State+N_Mesh
            R=1
        elif((State+10) in Obstacle_Four):
            State=State+N_Mesh
            R=0.6
        else:
            State=State+N_Mesh
            R=0.2
    return State,R
        
#————————————————————————————————————————————————————————————————————————————————————————————
def Run_Function():
    Q_Table=Initial_Q_Table (N_States,Actions)
    Step=0 
    for episode in np.arange(4000) :#跑20回合
        if(episode%500==0):
            print(Q_Table)
        print('episode is %d' % episode)
        if(np.random.rand()<0.4):
            State_=1 #定义起点 
            print("Normal_start:%d"%State_)
        else:
            State_=np.random.randint(low=5, high=100)
            print("Random_start:%d"%State_)
        Is_End=False
        while not Is_End:
            A=Select_Action(State_,Q_Table,Step)
#             print(State_)
#             print(State_)
#             print(A)
            Next_S,R=Next_State(State_,A)
            Q_Table_S_A=Q_Table.loc[[State_],[A]]
            if Next_S!='End':
                D_T=np.array((Q_Table.loc[[Next_S],:]).T)
                Q_Target=R+Beta*D_T.max() #Next_S状态中的可选动作中的最大Q值动作
                Step+=1
            else:
                Q_Target=R
                Step+=1
                Is_End=True #目的是结束循
#————————————————————————————————————————————————————————————————————————————画图程序
#             X_State,Y_State=Caculate_coordinate(State_)
#             if(Next_S)=='End':
#                 H=N_States
#             else:
#                 H=Next_S
#             X_Next_State,Y_Next_State=Caculate_coordinate(H)                      
#             if(Step==0):
#                 plt.scatter(X_State,Y_State)  
#             else:
#                 plt.scatter(X_State,Y_State) 
#                 plt.scatter(X_Next_State,Y_Next_State)
#                 plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
#————————————————————————————————————————————————————————————————————————————
            Q_Table.loc[[State_],[A]]+=Alpha*(Q_Target-Q_Table_S_A)
            State_=Next_S
        print(Step)
        print(Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*Step))
        #plt.show()
        #print(Q_Table)
    return Q_Table
#————————————————————————————————————————————————————————————————————————————————————————————————————
Q__Table=Run_Function()
Q__Table
Final_Q_Table=np.array(Q__Table)

In [4]:
#—————————————————————————————————————————————————————————————————————————————————————————————————————
def Plot_Final_Graph(Q_TABLE):
    fig=plt.figure()
    ax=fig.gca()
    ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
    ax.set_xticks(np.arange(0,(N_Mesh+1)))
    ax.set_yticks(np.arange(0,(N_Mesh+1)))
    plt.tick_params(labelsize=23)
    plt.xlabel('x',size=30)
    plt.ylabel('y',size=30)
    fig.suptitle('Result of grid map based on Q-network(three types of obstacles)', fontsize=20)
    plt.grid()
    State_Start=1
    State=State_Start
    Step=1
    while(State!=N_States):
        Action=np.argmax((Q_TABLE[State-1]))
        print('--------------------------------------------------')
        print(Actions[Action])
        print(State)
        if(Action in [0,1,2]):
             Next_State=State-1
        elif(Action in [3,4,5]):
             Next_State=State+1
        else:
             Next_State=State+N_Mesh
        Step+=1
        print(Next_State)
        X_State,Y_State=Caculate_coordinate(State)
        X_Next_State,Y_Next_State=Caculate_coordinate(Next_State)
        if(Step==0):
            plt.scatter(X_State,Y_State)  
        else:
            plt.scatter(X_State,Y_State) 
            plt.scatter(X_Next_State,Y_Next_State)
            plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
        for J in np.arange(len(Obstacle)):
            plt.scatter((Caculate_coordinate(Obstacle[J]))[0],(Caculate_coordinate(Obstacle[J]))[1],marker="x",color='k',s=300)
        for J in np.arange(len(Obstacle_Six)):
            plt.scatter((Caculate_coordinate(Obstacle_Six[J]))[0],(Caculate_coordinate(Obstacle_Six[J]))[1],marker="H",color='k',s=300)
        for J in np.arange(len(Obstacle_Four)):
            plt.scatter((Caculate_coordinate(Obstacle_Four[J]))[0],(Caculate_coordinate(Obstacle_Four[J]))[1],marker="s",color='k',s=300)
        #print(X_State,Y_State)
        #print(X_Next_State,Y_Next_State)
        State=Next_State
    plt.show()
#——————————————————————————————————————————————————————————————————————————————————————————————————————————
Plot_Final_Graph(Final_Q_Table)

--------------------------------------------------
up_Four
1
11
--------------------------------------------------
right_Four
11
12
--------------------------------------------------
up_Six
12
22
--------------------------------------------------
up_Three
22
32
--------------------------------------------------
up_Three
32
42
--------------------------------------------------
up_Four
42
52
--------------------------------------------------
up_Three
52
62
--------------------------------------------------
up_Six
62
72
--------------------------------------------------
up_Six
72
82
--------------------------------------------------
up_Six
82
92
--------------------------------------------------
right_Three
92
93
--------------------------------------------------
right_Three
93
94
--------------------------------------------------
right_Three
94
95
--------------------------------------------------
right_Three
95
96
--------------------------------------------------
right_Three
96
97
----